## Let's start by importing Modules

In [1]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer

C:\Users\Akash\anaconda3\envs\movie\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Importing the transformer model we are going to use

In [2]:
model = SentenceTransformer("all-MiniLM-L6-v2")

## loading the csv file

In [3]:
df=pd.read_csv('TMDB_movie_dataset_v11.csv')
df

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,...,original_title,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,keywords
0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,...,Inception,"Cobb, a skilled thief who commits corporate es...",83.952,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili","rescue, mission, dream, airplane, paris, franc..."
1,157336,Interstellar,8.417,32571,Released,2014-11-05,701729206,169,False,/pbrkL804c8yAv3zBZR4QPEafpAR.jpg,...,Interstellar,The adventures of a group of explorers who mak...,140.241,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,Mankind was born on Earth. It was never meant ...,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America",English,"rescue, future, spacecraft, race against time,..."
2,155,The Dark Knight,8.512,30619,Released,2008-07-16,1004558444,152,False,/nMKdUUepR0i5zn0y1T4CsSB5chy.jpg,...,The Dark Knight,Batman raises the stakes in his war on crime. ...,130.643,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,Welcome to a world without rules.,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","United Kingdom, United States of America","English, Mandarin","joker, sadism, chaos, secret identity, crime f..."
3,19995,Avatar,7.573,29815,Released,2009-12-15,2923706026,162,False,/vL5LR6WdxWPjLPFRLe133jXWsh5.jpg,...,Avatar,"In the 22nd century, a paraplegic Marine is di...",79.932,/kyeqWdyUXW608qlYkRqosgbbJyK.jpg,Enter the world of Pandora.,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","United States of America, United Kingdom","English, Spanish","future, society, culture clash, space travel, ..."
4,24428,The Avengers,7.710,29166,Released,2012-04-25,1518815515,143,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,...,The Avengers,When an unexpected enemy emerges and threatens...,98.082,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,Some assembly required.,"Science Fiction, Action, Adventure",Marvel Studios,United States of America,"English, Hindi, Russian","new york city, superhero, shield, based on com..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1208034,767713,Gli indocili,0.000,0,Released,2019-01-20,0,0,False,NaN,...,Gli indocili,NaN,0.600,/o2Q8ASR7KPYJfK9bMljRHdVSdps.jpg,NaN,NaN,NaN,NaN,NaN,NaN
1208035,767714,Carla,0.000,0,Released,1968-10-01,0,5,False,NaN,...,Carla,"Carla is a different form of homage, in which ...",0.603,/19mUjoSjEHcl8vq0wmkBpVaeI5z.jpg,NaN,NaN,NaN,Germany,NaN,NaN
1208036,767717,Fugue,0.000,0,Released,NaN,0,20,False,NaN,...,Fugue,"The metaphysical story, which has happened at ...",0.600,/rGid2uhAUHSXJ8SbK9t4XcnmMOp.jpg,NaN,NaN,NaN,NaN,NaN,NaN
1208037,767718,Starbound,0.000,0,Released,NaN,0,99,False,NaN,...,Starbound,The Starbound was built in 1989 at Dakota Cree...,0.600,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## We will have to preprocess the data.
1. We only need those columns which are required for content based recommender system(i.e. Overview, Genre,etc)
2. We have to clean the data(i.e. dropping N.A values)

In [4]:
df.head(1)

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,...,original_title,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,keywords
0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,...,Inception,"Cobb, a skilled thief who commits corporate es...",83.952,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili","rescue, mission, dream, airplane, paris, franc..."


In [5]:
df.columns

Index(['id', 'title', 'vote_average', 'vote_count', 'status', 'release_date',
       'revenue', 'runtime', 'adult', 'backdrop_path', 'budget', 'homepage',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'tagline', 'genres',
       'production_companies', 'production_countries', 'spoken_languages',
       'keywords'],
      dtype='object')

In [6]:
print(list(df.original_language.value_counts()))

[654473, 70648, 61814, 56559, 52693, 40872, 36088, 25183, 24841, 13913, 11180, 9457, 9103, 8964, 8242, 8207, 7442, 6748, 6003, 5608, 5183, 5124, 5106, 4829, 4204, 4140, 4010, 3494, 3455, 3414, 3026, 2935, 2637, 2578, 2513, 1844, 1666, 1556, 1515, 1407, 1358, 1353, 1345, 1251, 1243, 1214, 1157, 1116, 1073, 1033, 937, 881, 729, 725, 601, 584, 569, 539, 493, 480, 405, 393, 389, 363, 340, 318, 298, 269, 261, 240, 233, 214, 180, 157, 137, 120, 115, 114, 110, 108, 92, 92, 89, 68, 64, 62, 62, 56, 56, 56, 55, 51, 49, 48, 45, 44, 43, 42, 38, 38, 36, 36, 35, 34, 32, 31, 29, 27, 23, 23, 23, 21, 20, 20, 20, 17, 15, 15, 15, 15, 13, 12, 12, 12, 12, 11, 11, 11, 10, 10, 10, 9, 9, 9, 8, 7, 7, 7, 7, 7, 7, 6, 5, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [7]:
df=df.drop(['vote_average', 'vote_count','revenue', 'runtime', 'adult', 'budget', 'homepage','original_language', 'original_title','popularity','production_companies', 'production_countries', 'spoken_languages',
       'keywords','tagline','backdrop_path',"imdb_id", 'release_date','id'],axis=1)

In [8]:
df

,title,status,overview,poster_path,genres
0,Inception,Released,"Cobb, a skilled thief who commits corporate es...",/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,"Action, Science Fiction, Adventure"
1,Interstellar,Released,The adventures of a group of explorers who mak...,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,"Adventure, Drama, Science Fiction"
2,The Dark Knight,Released,Batman raises the stakes in his war on crime. ...,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,"Drama, Action, Crime, Thriller"
3,Avatar,Released,"In the 22nd century, a paraplegic Marine is di...",/kyeqWdyUXW608qlYkRqosgbbJyK.jpg,"Action, Adventure, Fantasy, Science Fiction"
4,The Avengers,Released,When an unexpected enemy emerges and threatens...,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,"Science Fiction, Action, Adventure"
...,...,...,...,...,...
1208034,Gli indocili,Released,NaN,/o2Q8ASR7KPYJfK9bMljRHdVSdps.jpg,NaN
1208035,Carla,Released,"Carla is a different form of homage, in which ...",/19mUjoSjEHcl8vq0wmkBpVaeI5z.jpg,NaN
1208036,Fugue,Released,"The metaphysical story, which has happened at ...",/rGid2uhAUHSXJ8SbK9t4XcnmMOp.jpg,NaN
1208037,Starbound,Released,The Starbound was built in 1989 at Dakota Cree...,NaN,NaN


In [9]:
df.isna().sum()

title              13
status              0
overview       254354
poster_path    394726
genres         497750
dtype: int64

In [10]:
df=df.dropna()

In [11]:
df

,title,status,overview,poster_path,genres
0,Inception,Released,"Cobb, a skilled thief who commits corporate es...",/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,"Action, Science Fiction, Adventure"
1,Interstellar,Released,The adventures of a group of explorers who mak...,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,"Adventure, Drama, Science Fiction"
2,The Dark Knight,Released,Batman raises the stakes in his war on crime. ...,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,"Drama, Action, Crime, Thriller"
3,Avatar,Released,"In the 22nd century, a paraplegic Marine is di...",/kyeqWdyUXW608qlYkRqosgbbJyK.jpg,"Action, Adventure, Fantasy, Science Fiction"
4,The Avengers,Released,When an unexpected enemy emerges and threatens...,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,"Science Fiction, Action, Adventure"
...,...,...,...,...,...
1208025,"He Leads, Others Follow",Released,"He Leads, Others Follow is a 1919 American sho...",/wsa78lBSIegotBfddtemsgQyy1f.jpg,Comedy
1208026,Soft Money,Released,Soft Money is a 1919 American short comedy fil...,/nfVz6VvXbhRgqEEQkeQl83pyrzS.jpg,Comedy
1208029,Schatten im Rampenlicht,Released,Shadows in the Spotlight: Stand-ins and bit pl...,/52jB38vvhNA0AiQNUxDwJSoOA2x.jpg,Documentary
1208031,Meu Amigo Alemão,Released,"Gerd Wenzel was born in Berlin, and as a boy h...",/c2KNkLc47iv6YblF6iQDXumjx9h.jpg,Documentary


## we only need those movies which are released

In [12]:
df=df[df['status'].str.lower() == 'released']

In [13]:
df=df.drop('status',axis=1)

In [14]:
df

,title,overview,poster_path,genres
0,Inception,"Cobb, a skilled thief who commits corporate es...",/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,"Action, Science Fiction, Adventure"
1,Interstellar,The adventures of a group of explorers who mak...,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,"Adventure, Drama, Science Fiction"
2,The Dark Knight,Batman raises the stakes in his war on crime. ...,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,"Drama, Action, Crime, Thriller"
3,Avatar,"In the 22nd century, a paraplegic Marine is di...",/kyeqWdyUXW608qlYkRqosgbbJyK.jpg,"Action, Adventure, Fantasy, Science Fiction"
4,The Avengers,When an unexpected enemy emerges and threatens...,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,"Science Fiction, Action, Adventure"
...,...,...,...,...
1208025,"He Leads, Others Follow","He Leads, Others Follow is a 1919 American sho...",/wsa78lBSIegotBfddtemsgQyy1f.jpg,Comedy
1208026,Soft Money,Soft Money is a 1919 American short comedy fil...,/nfVz6VvXbhRgqEEQkeQl83pyrzS.jpg,Comedy
1208029,Schatten im Rampenlicht,Shadows in the Spotlight: Stand-ins and bit pl...,/52jB38vvhNA0AiQNUxDwJSoOA2x.jpg,Documentary
1208031,Meu Amigo Alemão,"Gerd Wenzel was born in Berlin, and as a boy h...",/c2KNkLc47iv6YblF6iQDXumjx9h.jpg,Documentary


In [15]:
df=df.reset_index(drop=True)
df

,title,overview,poster_path,genres
0,Inception,"Cobb, a skilled thief who commits corporate es...",/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,"Action, Science Fiction, Adventure"
1,Interstellar,The adventures of a group of explorers who mak...,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,"Adventure, Drama, Science Fiction"
2,The Dark Knight,Batman raises the stakes in his war on crime. ...,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,"Drama, Action, Crime, Thriller"
3,Avatar,"In the 22nd century, a paraplegic Marine is di...",/kyeqWdyUXW608qlYkRqosgbbJyK.jpg,"Action, Adventure, Fantasy, Science Fiction"
4,The Avengers,When an unexpected enemy emerges and threatens...,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,"Science Fiction, Action, Adventure"
...,...,...,...,...
461753,"He Leads, Others Follow","He Leads, Others Follow is a 1919 American sho...",/wsa78lBSIegotBfddtemsgQyy1f.jpg,Comedy
461754,Soft Money,Soft Money is a 1919 American short comedy fil...,/nfVz6VvXbhRgqEEQkeQl83pyrzS.jpg,Comedy
461755,Schatten im Rampenlicht,Shadows in the Spotlight: Stand-ins and bit pl...,/52jB38vvhNA0AiQNUxDwJSoOA2x.jpg,Documentary
461756,Meu Amigo Alemão,"Gerd Wenzel was born in Berlin, and as a boy h...",/c2KNkLc47iv6YblF6iQDXumjx9h.jpg,Documentary


In [16]:
df.genres.head(1)

0    Action, Science Fiction, Adventure
Name: genres, dtype: object

## now we will create 1 column based on overview and genre for simplicity and efficiency

In [17]:
df['tags'] = df['overview'] + " " + df['genres']

In [18]:
df.tags=df.tags.str.lower()

In [19]:
df

,title,overview,poster_path,genres,tags
0,Inception,"Cobb, a skilled thief who commits corporate es...",/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,"Action, Science Fiction, Adventure","cobb, a skilled thief who commits corporate es..."
1,Interstellar,The adventures of a group of explorers who mak...,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,"Adventure, Drama, Science Fiction",the adventures of a group of explorers who mak...
2,The Dark Knight,Batman raises the stakes in his war on crime. ...,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,"Drama, Action, Crime, Thriller",batman raises the stakes in his war on crime. ...
3,Avatar,"In the 22nd century, a paraplegic Marine is di...",/kyeqWdyUXW608qlYkRqosgbbJyK.jpg,"Action, Adventure, Fantasy, Science Fiction","in the 22nd century, a paraplegic marine is di..."
4,The Avengers,When an unexpected enemy emerges and threatens...,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,"Science Fiction, Action, Adventure",when an unexpected enemy emerges and threatens...
...,...,...,...,...,...
461753,"He Leads, Others Follow","He Leads, Others Follow is a 1919 American sho...",/wsa78lBSIegotBfddtemsgQyy1f.jpg,Comedy,"he leads, others follow is a 1919 american sho..."
461754,Soft Money,Soft Money is a 1919 American short comedy fil...,/nfVz6VvXbhRgqEEQkeQl83pyrzS.jpg,Comedy,soft money is a 1919 american short comedy fil...
461755,Schatten im Rampenlicht,Shadows in the Spotlight: Stand-ins and bit pl...,/52jB38vvhNA0AiQNUxDwJSoOA2x.jpg,Documentary,shadows in the spotlight: stand-ins and bit pl...
461756,Meu Amigo Alemão,"Gerd Wenzel was born in Berlin, and as a boy h...",/c2KNkLc47iv6YblF6iQDXumjx9h.jpg,Documentary,"gerd wenzel was born in berlin, and as a boy h..."


In [20]:
print(df['tags'].head(10))


0    cobb, a skilled thief who commits corporate es...
1    the adventures of a group of explorers who mak...
2    batman raises the stakes in his war on crime. ...
3    in the 22nd century, a paraplegic marine is di...
4    when an unexpected enemy emerges and threatens...
5    the origin story of former special forces oper...
6    as the avengers and their allies have continue...
7    a ticking-time-bomb insomniac and a slippery s...
8    light years from earth, 26 years after being a...
9    a burger-loving hit man, his philosophical par...
Name: tags, dtype: object


In [21]:
df.drop(['genres','overview'],axis=1,inplace=True)

In [22]:
df

,title,poster_path,tags
0,Inception,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,"cobb, a skilled thief who commits corporate es..."
1,Interstellar,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,the adventures of a group of explorers who mak...
2,The Dark Knight,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,batman raises the stakes in his war on crime. ...
3,Avatar,/kyeqWdyUXW608qlYkRqosgbbJyK.jpg,"in the 22nd century, a paraplegic marine is di..."
4,The Avengers,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,when an unexpected enemy emerges and threatens...
...,...,...,...
461753,"He Leads, Others Follow",/wsa78lBSIegotBfddtemsgQyy1f.jpg,"he leads, others follow is a 1919 american sho..."
461754,Soft Money,/nfVz6VvXbhRgqEEQkeQl83pyrzS.jpg,soft money is a 1919 american short comedy fil...
461755,Schatten im Rampenlicht,/52jB38vvhNA0AiQNUxDwJSoOA2x.jpg,shadows in the spotlight: stand-ins and bit pl...
461756,Meu Amigo Alemão,/c2KNkLc47iv6YblF6iQDXumjx9h.jpg,"gerd wenzel was born in berlin, and as a boy h..."


In [23]:
print(len(df['tags'])) 


461758


In [24]:
print(df['tags'].isna().sum())


0


In [25]:
print(df.shape)


(461758, 3)


## Now we have to create embeddings and store it to a pickle file which will be used in another jupyter file. I have to comment out the steps after this because they are resource and time consuming.

In [26]:
# embeddings = model.encode(df['tags'].tolist(), batch_size=50, show_progress_bar=True)

In [27]:

# df['embeddings'] = list(embeddings)


In [28]:
# df

In [29]:
# df.to_pickle('movies.pkl')